# Query Transform and Routing Cookbook 

A user query can be transformed and decomposed in many ways before being executed as part of a RAG query engine, agent, or any other pipeline.

In this guide we show you different ways to transform, decompose, and rewrite queries. Each technique might be applicable for different use cases!

For naming purposes, we treat the underlying pipeline as a "tool".

1. **Sub-Questions**: Decompose queries into multiple sub-questions over different tools (identified by their metadata)
2. **Routing**: Keep the query, but identify the relevant subset of tools that the query applies to.
3. **Query-Rewriting**: Keep the tools, but rewrite the query in a variety of different ways to execute against the same tools.
4. **One-Step Query Transform**: Transform the query into another query given input prompt.

### Sub-Questions

Our first example is also the most complex one. Given a set of tools and a user query, decide both the 1) set of sub-questions to generate, and 2) the tools that each sub-question should run over.

In [2]:
from llama_index.question_gen import LLMQuestionGenerator, OpenAIQuestionGenerator
from llama_index.llms import OpenAI

In [3]:
llm = OpenAI()
question_gen = OpenAIQuestionGenerator.from_defaults(
    llm=llm
)

In [8]:
print(question_gen.get_prompts()["question_gen_prompt"].template)

You are a world class state of the art agent.

You have access to multiple tools, each representing a different data source or API.
Each of the tools has a name and a description, formatted as a JSON dictionary.
The keys of the dictionary are the names of the tools and the values are the descriptions.
Your purpose is to help answer a complex user question by generating a list of sub questions that can be answered by the tools.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the tools provided
* You can generate multiple sub questions for each tool
* Tools must be specified by their name, not their description
* You don't need to use a tool if you don't think it's relevant

Output the list of sub questions by calling the SubQuestionList function.

## Tools
```json
{tools_str}
```

## User Question
{query_str}



In [12]:
from llama_index.tools.types import ToolMetadata

tool_choices = [
    ToolMetadata(
        name="uber_2021_10k",
        description=(
            "Provides information about Uber financials for year 2021"
        ),
    ),
    ToolMetadata(
        name="lyft_2021_10k",
        description=(
            "Provides information about Lyft financials for year 2021"
        )
    )
]

In [ ]:
from llama_index.schema import QueryBundle

query_str = "Compare and contrast Uber and Lyft"
choices = question_gen.generate(tool_choices, QueryBundle(query_str=query_str))